In [ ]:
import os
from PIL import Image
from manga_ocr import MangaOcr
from tqdm import tqdm
import glob
import xml.etree.ElementTree as ET
import cv2
import matplotlib.pyplot as plt
import re
import unicodedata

In [ ]:
# Initialize MangaOcr
mocr = MangaOcr()

In [ ]:
# --- 1. Configuration Section ---
END_WITH_LOCAL = 'manga-ocr'

NOTEBOOK_DIR = os.getcwd()
print(f"NOTEBOOK_DIR: {NOTEBOOK_DIR}")

# Simple validation
if not (NOTEBOOK_DIR.endswith('/content') or NOTEBOOK_DIR.endswith(END_WITH_LOCAL)):
    raise ValueError(f"Expected to be in .../{END_WITH_LOCAL} or .../content directory, but got: {NOTEBOOK_DIR}")

BASE_DIR = os.path.join(NOTEBOOK_DIR, '..', '..', '..')
print(f"BASE_DIR: {BASE_DIR}")

# Define paths to the Manga109 dataset
IMAGE_ROOT_DIR = os.path.join(BASE_DIR, 'data', 'Manga109_released_2023_12_07/images')
ANNOTATIONS_DIR = os.path.join(BASE_DIR, 'data', 'Manga109_released_2023_12_07/annotations')

# Check if the directories exist
if not os.path.isdir(IMAGE_ROOT_DIR):
    print(f"Warning: Manga images root directory not found at '{IMAGE_ROOT_DIR}'")
    print("Please ensure the Manga109 dataset is downloaded and the path is correct.")
if not os.path.isdir(ANNOTATIONS_DIR):
    print(f"Warning: Annotations directory not found at '{ANNOTATIONS_DIR}'")

xml_files = sorted([f for f in os.listdir(ANNOTATIONS_DIR) if f.endswith('.xml')])
print(f"Found {len(xml_files)} manga books with annotations.")

In [ ]:
# --- 2. Process each book ---
def clean_manga_ocr_text(text):
    if not text:
        return ""

    # 1. Chuẩn hóa Unicode về dạng chuẩn (NFKC)
    text = unicodedata.normalize('NFKC', text)

    # 2. Xóa toàn bộ khoảng trắng (Space, Tab, Newline)
    text = re.sub(r'\s+', '', text)

    # 3. Gộp dấu chấm: Biến '.....' hoặc '..' thành '…'
    text = re.sub(r'\.{2,}', '…', text)

    # 4. Chuẩn hóa dấu ngã
    text = text.replace('~', '〜')

    return text

results = []

for xml_file in tqdm(xml_files, desc="Processing manga books"):
    book_name = os.path.splitext(xml_file)[0]
    xml_path = os.path.join(ANNOTATIONS_DIR, xml_file)
    
    book_image_dir = os.path.join(IMAGE_ROOT_DIR, book_name)
    if not os.path.isdir(book_image_dir):
        if '_vol' in book_name:
            base_book_name = book_name.split('_vol')[0]
            book_image_dir = os.path.join(IMAGE_ROOT_DIR, base_book_name)
            if not os.path.isdir(book_image_dir):
                print(f"Warning: Could not find image directory for book '{book_name}'")
                continue
        else:
            print(f"Warning: Could not find image directory for book '{book_name}'")
            continue

    try:
        tree = ET.parse(xml_path)
        root = tree.getroot()
        pages = root.find('pages')

        if pages is None:
            continue

        for page in tqdm(pages.findall('page'), desc=f"Processing pages in {book_name}", leave=False):
            page_index = int(page.get('index'))
            
            # Construct image path
            img_filename = f"{page_index}.jpg"
            img_path = os.path.join(book_image_dir, img_filename)

            if not os.path.exists(img_path):
                # Try with png extension as a fallback
                img_path = os.path.join(book_image_dir, f"{page_index}.png")
                if not os.path.exists(img_path):
                    continue

            try:
                image = cv2.imread(img_path)
                if image is None:
                    continue
                
                # Convert image to RGB for PIL
                image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

                for text_element in page.findall('.//text'):
                    text_id = text_element.get('id')
                    text = text_element.text
                    xmin = int(text_element.get('xmin'))
                    ymin = int(text_element.get('ymin'))
                    xmax = int(text_element.get('xmax'))
                    ymax = int(text_element.get('ymax'))

                    # Crop the bubble from the image
                    cropped_bubble = image_rgb[ymin:ymax, xmin:xmax]

                    if cropped_bubble.size == 0:
                        continue
                    
                    # Convert cropped numpy array to PIL Image
                    pil_image = Image.fromarray(cropped_bubble)
                    
                    # Perform OCR
                    raw_ocr_text = mocr(pil_image)
                    cleaned_ocr_text = clean_manga_ocr_text(raw_ocr_text)
                    
                    results.append({
                        'book': book_name,
                        'page': page_index,
                        'text_id': text_id,
                        'bbox': [xmin, ymin, xmax, ymax],
                        'ground_truth': text,
                        'ocr_result_raw': raw_ocr_text,
                        'ocr_result_cleaned': cleaned_ocr_text,
                        'image': pil_image
                    })

            except Exception as e:
                print(f"Error processing page {page_index} of {book_name}: {e}")

    except ET.ParseError as e:
        print(f"Error parsing XML file {xml_file}: {e}")

In [ ]:
# --- 3. Visualize some results ---
import random

def visualize_result(result):
    """Visualizes a single OCR result and prints its details."""
    print(f"Book: {result['book']}")
    print(f"Page: {result['page']}")
    print(f"Text ID: {result['text_id']}")
    print(f"Ground Truth: {result['ground_truth']}")
    print(f"OCR Result (Raw):     {result['ocr_result_raw']}")
    print(f"OCR Result (Cleaned): {result['ocr_result_cleaned']}")
    
    plt.imshow(result['image'])
    plt.axis('off')
    plt.show()

# Visualize a random result
if results:
    random_result = random.choice(results)
    visualize_result(random_result)
else:
    print("No results to display.")

In [ ]:
# --- 4. Save Raw results to JSON ---
import json

# Prepare results for JSON serialization
results_raw_to_save = []

for res in results:
    # Common fields
    base_res = {
        'book': res['book'],
        'page': res['page'],
        'text_id': res['text_id'],
        'bbox': res['bbox'],
        'ground_truth': res['ground_truth']
    }
    
    # Raw version
    res_raw = base_res.copy()
    res_raw['ocr_result'] = res['ocr_result_raw']
    results_raw_to_save.append(res_raw)

# Define output paths
output_dir = os.path.join(BASE_DIR, 'output', 'MangaOCR_inference')
os.makedirs(output_dir, exist_ok=True)

output_json_raw_path = os.path.join(output_dir, 'manga_ocr_results_raw.json')

# Save Raw
if results_raw_to_save:
    with open(output_json_raw_path, 'w', encoding='utf-8') as f:
        json.dump(results_raw_to_save, f, ensure_ascii=False, indent=4)
    print(f"Successfully saved raw results to {output_json_raw_path}")
else:
    print("No raw results to save.")

In [ ]:
# --- 5. Save Cleaned results to JSON ---
results_cleaned_to_save = []

for res in results:
    # Common fields
    base_res = {
        'book': res['book'],
        'page': res['page'],
        'text_id': res['text_id'],
        'bbox': res['bbox'],
        'ground_truth': res['ground_truth']
    }
    
    # Cleaned version
    res_cleaned = base_res.copy()
    res_cleaned['ocr_result'] = res['ocr_result_cleaned']
    results_cleaned_to_save.append(res_cleaned)

output_json_cleaned_path = os.path.join(output_dir, 'manga_ocr_results_cleaned.json')

# Save Cleaned
if results_cleaned_to_save:
    with open(output_json_cleaned_path, 'w', encoding='utf-8') as f:
        json.dump(results_cleaned_to_save, f, ensure_ascii=False, indent=4)
    print(f"Successfully saved cleaned results to {output_json_cleaned_path}")
else:
    print("No cleaned results to save.")

In [ ]:
# --- 6. Evaluate Raw Output ---
try:
    from torchmetrics.text import CharErrorRate, WordErrorRate
except ImportError:
    print("torchmetrics not found. Please install it using: pip install torchmetrics")

def evaluate_predictions(predictions, targets, label=""):
    if not predictions:
        print(f"No samples for {label}")
        return

    cer_metric = CharErrorRate()
    cer = cer_metric(predictions, targets)
    
    wer_metric = WordErrorRate()
    wer = wer_metric(predictions, targets)

    print(f"--- {label} ---")
    print(f"Character Error Rate (CER): {cer.item():.4f}")
    print(f"Word Error Rate (WER):      {wer.item():.4f}")

preds_raw = []
target = []

print("Preparing data for Raw evaluation...")
for res in results:
    # Always clean GT for comparison to ensure fair metrics
    gt = clean_manga_ocr_text(res['ground_truth'])
    
    if gt.strip():
        preds_raw.append(res['ocr_result_raw'])
        target.append(gt)

print(f"Evaluating on {len(target)} samples...")

if target:
    evaluate_predictions(preds_raw, target, "Raw Output")
else:
    print("No valid samples for evaluation.")

In [ ]:
# --- 7. Evaluate Cleaned Output ---
preds_cleaned = []
target = []

print("Preparing data for Cleaned evaluation...")
for res in results:
    # Always clean GT for comparison to ensure fair metrics
    gt = clean_manga_ocr_text(res['ground_truth'])
    
    if gt.strip():
        preds_cleaned.append(res['ocr_result_cleaned'])
        target.append(gt)

print(f"Evaluating on {len(target)} samples...")

if target:
    evaluate_predictions(preds_cleaned, target, "Post-processed Output")
    
    print("\n" + "="*40)
    print("Sample Discrepancies (Cleaned vs GT)")
    print("="*40)
    count = 0
    for cleaned, t in zip(preds_cleaned, target):
        if cleaned != t:
            print(f"GT:      {t}")
            print(f"Cleaned: {cleaned}")
            print("-" * 20)
            count += 1
            if count >= 10:
                break
else:
    print("No valid samples for evaluation.")